In [1]:
import pandas as pd
import numpy as np
import re
from re import search
import codecs
import sys
import string
import math
import heapq


In [2]:
stop_words = []
prefixes = []
postfixes = []
verb_roots = []
common_words = []
plural_singular = []

In [6]:
#######################################importing
def read_dataset(path, name):
    df = pd.read_excel(path + name)
    return df

def read_files():
    path = 'files/'

    #stop_words
    name = 'stop_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        stop_words.append(line)
    f.close()

    #prefixes
    name = 'prefixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        prefixes.append(line)
    f.close()

    #postfixes
    name = 'postfixes.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        postfixes.append(line)
    f.close()

    # verb_roots
    name = 'verb.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        verb_roots.append(line)
    f.close()

    #common_words
    name = 'common_words.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    # Strips the newline character
    for line in Lines:
        common_words.append(line)
    f.close()

    #plural_singular
    name = 'plural_singular.txt'
    f = open(path + name, 'r', encoding='utf-8')
    Lines = f.read().splitlines()
    count = 0
    # Strips the newline character
    for line in Lines:
        line = line.split('-')
        plural_singular.append(line)
    f.close()

#######################################Normalization
def delete_punctuations(doc):
    punctuations = '؟،:؛!»«()[]*,{.}@!?،؛<>#$&!~"\|-_+'
    edited_doc = doc.translate(str.maketrans('', '', punctuations))
    edited_doc = edited_doc.translate(str.maketrans('', '', string.punctuation))
    
    return edited_doc

def delete_stopWords(doc):
    edited_doc = doc
    for s in stop_words:
        my_regex = r"\b"+s+r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_highFrequencyWords(inverted_indexes):
    for i in inverted_indexes:
        if len(i[1])/len(df) > 0.8:
            inverted_indexes.remove(i)
    return inverted_indexes

def delete_postfixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = p + r"\b"
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def delete_prefixes(doc):
    edited_doc = doc
    for p in postfixes:
        my_regex = r"\b" + p
        edited_doc = re.sub(my_regex , "", edited_doc)
    return edited_doc

def replaceWithRoot(tokens): 
    for i in range(0, len(tokens)):
        # print(verb_roots)
        for root in verb_roots:
            if search(root[0], tokens[i]) or search(root[1], tokens[i]):
                common = False
                for c in common_words:
                    if c == tokens[i]:
                        common = True
                        # print("common")
                        break
                if common == False:
                    my_regex = r"\b^"+root[1]+r"\.*"
                    x = re.findall(my_regex, tokens[i])
 
                    if len(x) == 0:  
                        my_regex = r"\b...*"+root[1]+r"\b"
                        x1 = re.findall(my_regex, tokens[i])
                        if len(x1)==0:         
                            # print("root")
                            tokens[i] = tokens[i].replace(tokens[i], root[2])

    return tokens

def replaceArabicWords(doc):
    doc = doc.replace('ك', 'ک')
    doc = doc.replace('ئ', 'ی')
    doc = doc.replace('ي', 'ی')
    doc = doc.replace('ؤ', 'و')
    doc = doc.replace('هٔ', 'ه')
    doc = doc.replace('ة', 'ه')
    doc = doc.replace('آ', 'ا')
    doc = doc.replace('أ', 'ا')
    doc = doc.replace('إ', 'ا')
    return doc

def pluralToSingular(tokens):
    for i in range(0, len(tokens)):
        for ps in plural_singular:
            if search(ps[0], tokens[i]):
                tokens[i] = tokens[i].replace(tokens[i], ps[1])
    return tokens

#######################################tokenization
def tokenize(df):
    content = df.content

    tokens = []
    for i in range(0, content.size):
        doc = content[i]
        # 68000 tokens
        doc = delete_punctuations(doc)
        # 50000 tokens
        doc = delete_stopWords(doc)

        doc = replaceArabicWords(doc)

        doc = delete_postfixes(doc)

        doc = delete_prefixes(doc)
        

        tokenized_doc = doc.split()
        tokenized_doc = replaceWithRoot(tokenized_doc)
        tokenized_doc = pluralToSingular(tokenized_doc)

        for token in tokenized_doc:
            if len(token) > 0:
                temp = []
                temp.append(token)
                temp.append(df.id[i])
                tokens.append(temp)
    tokens.sort()
    return tokens

#######################################create inverted index
def create_inverted_indexes(tokens):
    inverted_indexes = []
    doc_temp = []
    token_temp = ""
    for token in tokens:
        if token[0] == token_temp:
            doc_temp.append(token[1])
        else:
            temp = []
            temp.append(token_temp)

            (unique, counts) = np.unique(doc_temp, return_counts=True)
            # frequencies = np.array((unique, counts), dtype='i4,f4').T.view(np.recarray)       
            frequencies = []      
            for i in range(len(unique)):
                temp1 = []
                temp1.append(unique[i])
                temp1.append(counts[i])
                frequencies.append(temp1)
            temp.append(frequencies)
            inverted_indexes.append(temp)
            doc_temp = []

        token_temp = token[0]
        # if len(doc_temp) > 0 and token[1] != doc_temp[-1]:
        doc_temp.append(token[1])
    
    inverted_indexes = delete_highFrequencyWords(inverted_indexes)
    return inverted_indexes

#######################################calculate weights
def calculate_tfidf(df, inverted_indexes):
    doc_length = np.zeros(len(df)+1)
    len_df = len(df)
    for i in range(len(inverted_indexes)):
        if len(inverted_indexes[i][1]) > 0:
            # print(weighted_inverted_index[i])
            # print(len(inverted_indexes[i][1]))
            idf = math.log(len_df/len(inverted_indexes[i][1]), 10)
            inverted_indexes[i].append(idf)
            # print("idf = ", idf)
            for j in range(len(inverted_indexes[i][1])):
                tf = 1 + math.log(inverted_indexes[i][1][j][1], 10)
                # print("tf = ", tf)
                w = tf*idf
                # print("w : ",w)
                inverted_indexes[i][1][j][1] = w
                doc_length[inverted_indexes[i][1][j][0]] = doc_length[inverted_indexes[i][1][j][0]] + np.power(w, 2)
                
    doc_length =  np.sqrt(doc_length)
    return inverted_indexes, doc_length

#######################################create champion list
def create_championList(inverted_indexes, r):
    limit = r
    championList = []
    for i in range(len(inverted_indexes)):
        temp = []
        temp.append(inverted_indexes[i][0])
        temp_postingList = list(inverted_indexes[i][1])
        temp_postingList.sort(key=lambda x: x[1], reverse=True)

        temp1 = []
        r = limit
        if r > len(temp_postingList):
            r = len(temp_postingList)
        for j in range(r):
            # print(temp_postingList[j])
            temp1.append(temp_postingList[j])
        temp.append(temp1)
        if len(inverted_indexes[i]) == 3:
            temp.append(inverted_indexes[i][2])
        championList.append(temp)
    return championList

#######################################Searching
def search_token(inverted_indexes, token_name):
    for token in inverted_indexes:
        if token[0] == token_name:
            print("token*****: ", token[0])
            return token[1], token[2]
    
    print("this token not exist in our database")
    return -1, -1

def print_res(df, res):
    p_res = pd.DataFrame()

    p_res = pd.merge(df, res, on=['id'], how='inner')
    our_res = p_res[['id', 'rank', 'url']]
    our_res = our_res.sort_values(["rank"], ascending=False)

    print(our_res)
    

def binary_search(df, inverted_indexes, query):
    res = []
    docs_id = pd.DataFrame()


    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return


    for i in range (0 ,len(tokenized_query)):
        docs, idf = search_token(inverted_indexes, tokenized_query[i])
        if docs != -1:
            temp_docs_id = pd.DataFrame(docs)
            docs_id = pd.DataFrame(docs_id.append(temp_docs_id))

    if len(docs_id) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1

    docs_id = docs_id[0].value_counts().reset_index()
    docs_id.columns = ['id', 'rank']
    print_res(df, docs_id)


def ranked_search(df, inverted_indexes, query, doc_length, k, using_heap):
    res = []
    docs_id = pd.DataFrame()
    
    query = delete_punctuations(query)
    query = delete_stopWords(query)
    query = delete_postfixes(query)
    query = delete_prefixes(query)
    query = replaceArabicWords(query)

    tokenized_query = query.split()
    tokenized_query = replaceWithRoot(tokenized_query)
    tokenized_query = pluralToSingular(tokenized_query)

    if len(tokenized_query) == 0:
        print("It looks like there aren't many great matches for your search")
        return -1

    (unique, counts) = np.unique(tokenized_query, return_counts=True)

    tokenized_query = np.asarray((unique, counts)).T
    # print(tokenized_query)

    doc_score_temp = []
    for i in range (0 ,len(tokenized_query)):
        tf_query = 1 + math.log(int(tokenized_query[i][1]), 10)
        # print(tokenized_query[i][0])
        docs, idf = search_token(inverted_indexes, tokenized_query[i][0])
        # print(docs)


        if docs != -1:
            # print(docs)
            # print(idf)
            w_termInQuery = tf_query*idf
            # print("weight term in q: ", tf_query, idf, w_termInQuery)

            for i in range(len(docs)):
                temp = []
                temp.append(docs[i][0])
                # print("docs[i][1]: ",docs[i][1])
                temp.append(w_termInQuery*docs[i][1])
                doc_score_temp.append(temp)

    if len(doc_score_temp) < 1:
        print("It looks like there aren't many great matches for your search")
        return -1
    doc_score_temp.sort()
    doc_score = []
    doc_score.append(doc_score_temp[0])
    for i in range(1, len(doc_score_temp)):
        if doc_score[len(doc_score)-1][0] == doc_score_temp[i][0]:
            doc_score[len(doc_score)-1][1] = doc_score[len(doc_score)-1][1] + doc_score_temp[i][1]
        else:
            doc_score.append(doc_score_temp[i])
    
    for i in range(len(doc_score)):
        doc_score[i][1] = doc_score[i][1] / doc_length[doc_score[i][0]]

    res = []
    if using_heap == False:
        doc_score.sort(key=lambda x: x[1], reverse=True)  
        if len(doc_score) > k:
            for i in range (k):
                res.append(doc_score[i])
        else:
            res = doc_score
    if using_heap == True:
        hp = []
        for e in doc_score:
            hp.append((e[1], e[0]))

        nlargest = heapq.nlargest(k, hp)
        k_high_ranked = []
        for i in nlargest:
            k_high_ranked.append([i[1], i[0]])

        res = k_high_ranked

    docs_id = pd.DataFrame(res, columns=['id', 'rank'])
    print_res(df, docs_id)


In [7]:
df = read_dataset("datasets/", "IR_Spring2021_ph12_7k.xlsx")
read_files()

In [11]:
tokens = tokenize(df)

In [12]:
inverted_indexes = create_inverted_indexes(tokens)

In [13]:
inverted_indexes, doc_length = calculate_tfidf(df, inverted_indexes)

In [14]:
championList = create_championList(inverted_indexes, 4)

In [28]:
query = "می‌داند"
# query_processing(df, inverted_indexes, query)

doc_score = ranked_search(df, championList, query, doc_length, k = 10, using_heap = True)
# doc_score = ranked_search(df, championList, query, doc_length, k = 10, using_heap = False)

doc_score = ranked_search(df, inverted_indexes, query, doc_length, k = 10, using_heap = True)

binary_search(df, inverted_indexes, query)

token*****:  دانستن
     id      rank                                                url
0  1041  0.018505  https://www.isna.ir/news/98010501325/سال-پر-فر...
3  5586  0.017072  https://www.isna.ir/news/99022316648/انستیتوپا...
2  2714  0.016354  https://www.isna.ir/news/98042110841/امام-جمعه...
1  2704  0.013673  https://www.isna.ir/news/98041407132/امام-جمعه...
token*****:  دانستن
     id      rank                                                url
1   334  0.106485  https://www.isna.ir/news/99050806176/ثبت-نام-ا...
0   330  0.098391  https://www.isna.ir/news/99050705189/مجید-کیها...
9  2016  0.097727  https://www.isna.ir/news/99060100664/نوروزی-وض...
8  1736  0.093934  https://www.isna.ir/news/99010803954/تعدادی-از...
3   831  0.090046  https://www.isna.ir/news/99102015020/اعزام-دوو...
2   351  0.087752  https://www.isna.ir/news/99051511624/ثبت-نام-پ...
6  1685  0.084743  https://www.isna.ir/news/98121512317/لغو-مسابق...
7  1722  0.084498  https://www.isna.ir/news/98122720984/ابتلای-

In [22]:
s = ['میرو']

print(replaceWithRoot(s))

['میرو']


In [24]:
print(len(inverted_indexes))
print(len(championList))

43019
43019
